# 🌱 Lemmatisation Française : Guide Complet

**Module 2 - Preprocessing et Tokenisation**

Ce notebook vous apprend tout sur la lemmatisation en français : concepts, outils, défis spécifiques et comparaison avec le stemming. Maîtrisez cette technique essentielle pour un preprocessing de qualité !

## 📋 Plan du Notebook

1. **Lemmatisation vs Stemming : Les Concepts**
2. **Spécificités du Français**
3. **Lemmatisation avec spaCy**
4. **Stemming en Français**
5. **Comparaison Détaillée**
6. **Cas Difficiles et Solutions**
7. **Lemmatiseur Personnalisé**
8. **Tests et Évaluation**
9. **Recommandations Pratiques**

## 📦 Installation et Imports

In [ ]:
# Installation des packages nécessaires
# !pip install spacy nltk matplotlib seaborn pandas wordcloud
# !python -m spacy download fr_core_news_sm
# !python -m spacy download fr_core_news_md  # Modèle plus précis (optionnel)

In [ ]:
import spacy
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import re
import time
from typing import List, Dict, Tuple, Set
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("Set1")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("✅ Imports réalisés avec succès !")

In [ ]:
# Chargement des modèles et ressources
print("🔄 Chargement des modèles de traitement...")

# spaCy français
try:
    nlp_fr = spacy.load("fr_core_news_sm")
    print(f"✅ spaCy français chargé : {nlp_fr.meta['name']} v{nlp_fr.meta['version']}")
except OSError:
    print("❌ Modèle spaCy français non trouvé")
    print("   Exécutez : python -m spacy download fr_core_news_sm")
    nlp_fr = None

# NLTK pour le stemming
nltk.download('punkt', quiet=True)
try:
    from nltk.stem import SnowballStemmer
    stemmer_fr = SnowballStemmer('french')
    print("✅ Stemmer français NLTK chargé")
except:
    stemmer_fr = None
    print("❌ Problème avec le stemmer NLTK")

print("\n🎯 Prêt pour la lemmatisation française !")

## 1. 🤔 Lemmatisation vs Stemming : Les Concepts

### Définitions

**🌱 Lemmatisation** : Réduction d'un mot à sa **forme canonique** (lemme) en utilisant un dictionnaire et des règles grammaticales.

**✂️ Stemming** : Suppression **mécanique** des suffixes pour obtenir la racine du mot.

### Exemple illustratif :

In [ ]:
# Démonstration simple des concepts
exemples_demonstration = [
    "mangeons", "mangeait", "mangeur", "mangeable",
    "courions", "courait", "coureur", "course",
    "finissons", "finissait", "finisseur", "finition",
    "parlons", "parlait", "parleur", "parole"
]

def demo_lemma_vs_stem(mots: List[str]) -> None:
    """
    Démonstration comparative lemmatisation vs stemming.
    """
    print("🔬 DÉMONSTRATION : LEMMATISATION vs STEMMING\n")
    print(f"{'Mot Original':15s} {'Lemme (spaCy)':15s} {'Stem (NLTK)':15s} {'Différence':15s}")
    print("="*70)
    
    for mot in mots:
        # Lemmatisation avec spaCy
        if nlp_fr:
            doc = nlp_fr(mot)
            lemme = doc[0].lemma_ if len(doc) > 0 else mot
        else:
            lemme = "N/A"
        
        # Stemming avec NLTK
        if stemmer_fr:
            stem = stemmer_fr.stem(mot)
        else:
            stem = "N/A"
        
        # Analyse de la différence
        if lemme != "N/A" and stem != "N/A":
            if lemme == stem:
                diff = "Identique"
            elif len(lemme) > len(stem):
                diff = "Lemme > Stem"
            else:
                diff = "Lemme < Stem"
        else:
            diff = "N/A"
        
        print(f"{mot:15s} {lemme:15s} {stem:15s} {diff:15s}")

# Exécution de la démonstration
demo_lemma_vs_stem(exemples_demonstration)

### 📊 Analyse de la démonstration :

- **Lemmatisation** : Produit des mots **valides** du français ("manger", "courir", "finir")
- **Stemming** : Produit des **racines** parfois non-mots ("mang", "cour", "fin")
- **Précision** : La lemmatisation est plus précise mais plus coûteuse
- **Vitesse** : Le stemming est plus rapide mais moins précis

## 2. 🇫🇷 Spécificités du Français

Le français présente des défis particuliers pour la lemmatisation :

In [ ]:
# Défis spécifiques du français
defis_francais = {
    "Verbes irréguliers": {
        "exemples": ["vais", "va", "irai", "ira", "suis", "est", "était", "fus"],
        "lemme_attendu": ["aller", "aller", "aller", "aller", "être", "être", "être", "être"],
        "difficulte": "Formes très différentes du lemme"
    },
    
    "Homonymie": {
        "exemples": ["fils", "fils", "suis", "suis"],
        "lemme_attendu": ["fil", "fils", "suivre", "être"],
        "difficulte": "Même forme, lemmes différents selon le contexte"
    },
    
    "Accords complexes": {
        "exemples": ["mangées", "finies", "prises", "venues"],
        "lemme_attendu": ["manger", "finir", "prendre", "venir"],
        "difficulte": "Accords du participe passé"
    },
    
    "Contractions": {
        "exemples": ["aujourd'hui", "quelqu'un", "lorsqu'il", "puisqu'elle"],
        "lemme_attendu": ["aujourd'hui", "quelqu'un", "lorsque", "puisque"],
        "difficulte": "Élisions et contractions figées"
    }
}

def tester_defis_francais() -> None:
    """
    Teste la lemmatisation sur les défis spécifiques du français.
    """
    print("🇫🇷 DÉFIS SPÉCIFIQUES DU FRANÇAIS\n")
    
    for categorie, info in defis_francais.items():
        print(f"📂 {categorie.upper()}")
        print(f"   💡 Difficulté : {info['difficulte']}")
        print(f"   🔍 Tests :")
        
        for i, mot in enumerate(info['exemples']):
            attendu = info['lemme_attendu'][i]
            
            if nlp_fr:
                doc = nlp_fr(mot)
                obtenu = doc[0].lemma_ if len(doc) > 0 else "Erreur"
                
                # Vérification
                if obtenu.lower() == attendu.lower():
                    status = "✅"
                else:
                    status = "❌"
                
                print(f"      {mot:12s} → {obtenu:12s} (attendu: {attendu:10s}) {status}")
            else:
                print(f"      {mot:12s} → N/A (spaCy non disponible)")
        
        print()

# Test des défis français
tester_defis_francais()

## 3. 🚀 Lemmatisation avec spaCy

spaCy est l'outil de référence pour la lemmatisation française de qualité.

In [ ]:
class LemmatiseurSpacy:
    """Classe pour la lemmatisation française avec spaCy."""
    
    def __init__(self, modele: str = "fr_core_news_sm"):
        """
        Initialise le lemmatiseur.
        
        Args:
            modele: Nom du modèle spaCy à utiliser
        """
        try:
            self.nlp = spacy.load(modele)
            self.modele_charge = True
            print(f"✅ Lemmatiseur spaCy initialisé avec {modele}")
        except OSError:
            self.nlp = None
            self.modele_charge = False
            print(f"❌ Modèle {modele} non trouvé")
    
    def lemmatiser_mot(self, mot: str) -> str:
        """
        Lemmatise un mot unique.
        
        Args:
            mot: Mot à lemmatiser
        
        Returns:
            Lemme du mot
        """
        if not self.modele_charge:
            return mot
        
        doc = self.nlp(mot)
        return doc[0].lemma_ if len(doc) > 0 else mot
    
    def lemmatiser_texte(self, texte: str, garder_pos: List[str] = None) -> List[str]:
        """
        Lemmatise un texte complet.
        
        Args:
            texte: Texte à lemmatiser
            garder_pos: Types de mots à conserver (NOUN, VERB, ADJ, etc.)
        
        Returns:
            Liste des lemmes
        """
        if not self.modele_charge:
            return texte.split()
        
        doc = self.nlp(texte)
        lemmes = []
        
        for token in doc:
            # Filtrage par type de mot si spécifié
            if garder_pos and token.pos_ not in garder_pos:
                continue
            
            # Ignorer la ponctuation et les espaces
            if not token.is_punct and not token.is_space:
                lemmes.append(token.lemma_.lower())
        
        return lemmes
    
    def analyse_detaillee(self, texte: str) -> List[Dict]:
        """
        Analyse détaillée avec informations grammaticales.
        
        Args:
            texte: Texte à analyser
        
        Returns:
            Liste de dictionnaires avec les analyses
        """
        if not self.modele_charge:
            return []
        
        doc = self.nlp(texte)
        analyses = []
        
        for token in doc:
            if not token.is_punct and not token.is_space:
                analyses.append({
                    'mot': token.text,
                    'lemme': token.lemma_,
                    'pos': token.pos_,
                    'tag': token.tag_,
                    'description': spacy.explain(token.tag_) or "Non défini",
                    'est_alpha': token.is_alpha,
                    'est_stopword': token.is_stop
                })
        
        return analyses
    
    def statistiques_lemmatisation(self, texte: str) -> Dict:
        """
        Calcule des statistiques sur la lemmatisation.
        
        Args:
            texte: Texte à analyser
        
        Returns:
            Dictionnaire avec les statistiques
        """
        if not self.modele_charge:
            return {}
        
        analyse = self.analyse_detaillee(texte)
        
        # Mots uniques avant/après lemmatisation
        mots_originaux = {item['mot'].lower() for item in analyse}
        lemmes_uniques = {item['lemme'].lower() for item in analyse}
        
        # Réduction du vocabulaire
        reduction = (len(mots_originaux) - len(lemmes_uniques)) / len(mots_originaux) * 100 if mots_originaux else 0
        
        # Distribution par type de mot
        pos_counts = Counter(item['pos'] for item in analyse)
        
        return {
            'mots_total': len(analyse),
            'mots_uniques_avant': len(mots_originaux),
            'lemmes_uniques_apres': len(lemmes_uniques),
            'reduction_vocabulaire_pct': round(reduction, 1),
            'distribution_pos': dict(pos_counts),
            'mots_changes': sum(1 for item in analyse if item['mot'].lower() != item['lemme'].lower()),
            'pourcentage_changes': round(sum(1 for item in analyse if item['mot'].lower() != item['lemme'].lower()) / len(analyse) * 100, 1) if analyse else 0
        }

# Test du lemmatiseur spaCy
lemmatiseur = LemmatiseurSpacy()

if lemmatiseur.modele_charge:
    # Test sur une phrase d'exemple
    phrase_test = "Les enfants couraient rapidement vers leurs parents en criant joyeusement."
    
    print("\n🧪 TEST DU LEMMATISEUR SPACY\n")
    print(f"📝 Phrase originale : {phrase_test}")
    
    # Lemmatisation simple
    lemmes = lemmatiseur.lemmatiser_texte(phrase_test)
    print(f"🌱 Lemmes : {' '.join(lemmes)}")
    
    # Statistiques
    stats = lemmatiseur.statistiques_lemmatisation(phrase_test)
    print(f"\n📊 Statistiques :")
    print(f"  • Mots traités : {stats['mots_total']}")
    print(f"  • Mots changés : {stats['mots_changes']} ({stats['pourcentage_changes']}%)")
    print(f"  • Réduction vocabulaire : {stats['reduction_vocabulaire_pct']}%")
else:
    print("⚠️ Lemmatiseur spaCy non disponible pour les tests")

In [ ]:
# Analyse détaillée d'un texte
if lemmatiseur.modele_charge:
    texte_analyse = "Les musiciens jouaient magnifiquement, les spectateurs applaudissaient enthousiastement."
    
    analyse_complete = lemmatiseur.analyse_detaillee(texte_analyse)
    
    print("🔬 ANALYSE GRAMMATICALE DÉTAILLÉE\n")
    print(f"📝 Texte : {texte_analyse}\n")
    
    # Création du tableau d'analyse
    df_analyse = pd.DataFrame(analyse_complete)
    
    # Sélection des colonnes importantes
    colonnes_affichage = ['mot', 'lemme', 'pos', 'description']
    df_affichage = df_analyse[colonnes_affichage].copy()
    
    # Ajout d'une colonne indiquant si le mot a changé
    df_affichage['changé'] = df_analyse['mot'].str.lower() != df_analyse['lemme'].str.lower()
    
    print("📋 Analyse token par token :")
    print(df_affichage.to_string(index=False))
    
    # Statistiques par type de mot
    print("\n📊 Distribution par type de mot :")
    pos_stats = df_analyse['pos'].value_counts()
    for pos, count in pos_stats.items():
        print(f"  {pos:8s} : {count:2d} ({count/len(df_analyse)*100:.1f}%)")
else:
    print("⚠️ Analyse détaillée non disponible (spaCy requis)")

## 4. ✂️ Stemming en Français

Le stemming comme alternative rapide à la lemmatisation.

In [ ]:
class StemmerFrancais:
    """Classe pour le stemming français avec différents algorithmes."""
    
    def __init__(self):
        """
        Initialise les stemmers disponibles.
        """
        # Stemmer NLTK (Snowball)
        try:
            self.stemmer_snowball = SnowballStemmer('french')
            self.snowball_disponible = True
        except:
            self.stemmer_snowball = None
            self.snowball_disponible = False
        
        # Stemmer simple basique (règles manuelles)
        self.suffixes_fr = [
            'ement', 'ation', 'ition', 'tion', 'sion',
            'ment', 'able', 'ible', 'ique', 'isme',
            'iste', 'eur', 'euse', 'ant', 'ent',
            'ons', 'ent', 'ait', 'ais', 'ez',
            'er', 'ir', 'es', 'e', 's'
        ]
        
        print(f"✅ StemmerFrancais initialisé")
        print(f"   Snowball : {'✅' if self.snowball_disponible else '❌'}")
        print(f"   Stemmer simple : ✅")
    
    def stem_snowball(self, mot: str) -> str:
        """
        Stemming avec l'algorithme Snowball.
        
        Args:
            mot: Mot à stemmer
        
        Returns:
            Radical du mot
        """
        if self.snowball_disponible:
            return self.stemmer_snowball.stem(mot)
        return mot
    
    def stem_simple(self, mot: str, longueur_min: int = 3) -> str:
        """
        Stemming simple par suppression de suffixes.
        
        Args:
            mot: Mot à stemmer
            longueur_min: Longueur minimale du radical
        
        Returns:
            Radical du mot
        """
        mot_lower = mot.lower()
        
        for suffixe in self.suffixes_fr:
            if mot_lower.endswith(suffixe) and len(mot_lower) - len(suffixe) >= longueur_min:
                return mot_lower[:-len(suffixe)]
        
        return mot_lower
    
    def stem_texte(self, texte: str, methode: str = 'snowball') -> List[str]:
        """
        Stemme un texte complet.
        
        Args:
            texte: Texte à stemmer
            methode: Méthode ('snowball' ou 'simple')
        
        Returns:
            Liste des radicaux
        """
        # Tokenisation simple
        mots = re.findall(r'\w+', texte.lower())
        
        if methode == 'snowball' and self.snowball_disponible:
            return [self.stem_snowball(mot) for mot in mots]
        else:
            return [self.stem_simple(mot) for mot in mots]
    
    def comparer_methodes(self, mots: List[str]) -> pd.DataFrame:
        """
        Compare les différentes méthodes de stemming.
        
        Args:
            mots: Liste de mots à tester
        
        Returns:
            DataFrame avec les comparaisons
        """
        resultats = []
        
        for mot in mots:
            resultat = {
                'mot_original': mot,
                'snowball': self.stem_snowball(mot) if self.snowball_disponible else 'N/A',
                'simple': self.stem_simple(mot)
            }
            resultats.append(resultat)
        
        return pd.DataFrame(resultats)

# Test du stemmer français
stemmer = StemmerFrancais()

# Mots de test
mots_test_stem = [
    "mangeons", "mangeait", "mangeable", "finissement",
    "courions", "courait", "rapidement", "lentement",
    "magnifique", "magnifiquement", "musiciens", "spectateurs"
]

print("\n✂️ TEST DU STEMMING FRANÇAIS\n")

# Comparaison des méthodes
if stemmer.snowball_disponible or True:  # Le stemmer simple est toujours disponible
    comparaison = stemmer.comparer_methodes(mots_test_stem)
    print("📊 Comparaison des méthodes de stemming :")
    print(comparaison.to_string(index=False))
    
    # Analyse des différences
    if stemmer.snowball_disponible:
        differences = sum(1 for _, row in comparaison.iterrows() 
                         if row['snowball'] != row['simple'])
        print(f"\n🔍 Différences entre méthodes : {differences}/{len(mots_test_stem)} mots")
else:
    print("⚠️ Aucun stemmer disponible pour les tests")

## 5. ⚔️ Comparaison Détaillée : Lemmatisation vs Stemming

Comparons les performances, la précision et l'usage des deux approches.

In [ ]:
class ComparateurLemmaStem:
    """Classe pour comparer lemmatisation et stemming."""
    
    def __init__(self):
        self.lemmatiseur = LemmatiseurSpacy()
        self.stemmer = StemmerFrancais()
    
    def test_performance(self, texte: str, nb_iterations: int = 10) -> Dict:
        """
        Teste les performances temporelles.
        
        Args:
            texte: Texte de test
            nb_iterations: Nombre d'itérations pour la mesure
        
        Returns:
            Dictionnaire avec les temps de traitement
        """
        # Test lemmatisation
        if self.lemmatiseur.modele_charge:
            start_time = time.time()
            for _ in range(nb_iterations):
                self.lemmatiseur.lemmatiser_texte(texte)
            temps_lemma = (time.time() - start_time) / nb_iterations
        else:
            temps_lemma = 0
        
        # Test stemming
        start_time = time.time()
        for _ in range(nb_iterations):
            self.stemmer.stem_texte(texte)
        temps_stem = (time.time() - start_time) / nb_iterations
        
        return {
            'temps_lemmatisation': temps_lemma,
            'temps_stemming': temps_stem,
            'ratio_vitesse': temps_lemma / temps_stem if temps_stem > 0 else 0
        }
    
    def test_qualite(self, texte: str) -> Dict:
        """
        Teste la qualité des résultats.
        
        Args:
            texte: Texte de test
        
        Returns:
            Dictionnaire avec les métriques de qualité
        """
        # Extraction des mots
        mots_originaux = re.findall(r'\w+', texte.lower())
        
        # Lemmatisation
        if self.lemmatiseur.modele_charge:
            lemmes = self.lemmatiseur.lemmatiser_texte(texte)
            vocabulaire_lemma = len(set(lemmes))
            reduction_lemma = (len(set(mots_originaux)) - vocabulaire_lemma) / len(set(mots_originaux)) * 100
        else:
            lemmes = []
            vocabulaire_lemma = 0
            reduction_lemma = 0
        
        # Stemming
        stems = self.stemmer.stem_texte(texte)
        vocabulaire_stem = len(set(stems))
        reduction_stem = (len(set(mots_originaux)) - vocabulaire_stem) / len(set(mots_originaux)) * 100
        
        return {
            'mots_originaux': len(set(mots_originaux)),
            'vocabulaire_lemmatisation': vocabulaire_lemma,
            'vocabulaire_stemming': vocabulaire_stem,
            'reduction_lemmatisation': round(reduction_lemma, 1),
            'reduction_stemming': round(reduction_stem, 1),
            'echantillon_lemmes': lemmes[:10],
            'echantillon_stems': stems[:10]
        }
    
    def test_lisibilite(self, mots: List[str]) -> Dict:
        """
        Teste la lisibilité des résultats (mots valides du français).
        
        Args:
            mots: Liste de mots à tester
        
        Returns:
            Dictionnaire avec les scores de lisibilité
        """
        resultats = {'mot': [], 'lemme': [], 'stem': [], 'lemme_valide': [], 'stem_valide': []}
        
        for mot in mots:
            # Lemmatisation
            if self.lemmatiseur.modele_charge:
                lemme = self.lemmatiseur.lemmatiser_mot(mot)
                lemme_valide = self._mot_francais_valide(lemme)
            else:
                lemme = mot
                lemme_valide = True
            
            # Stemming
            stem = self.stemmer.stem_snowball(mot)
            stem_valide = self._mot_francais_valide(stem)
            
            resultats['mot'].append(mot)
            resultats['lemme'].append(lemme)
            resultats['stem'].append(stem)
            resultats['lemme_valide'].append(lemme_valide)
            resultats['stem_valide'].append(stem_valide)
        
        # Calcul des scores
        score_lemme = sum(resultats['lemme_valide']) / len(resultats['lemme_valide']) * 100
        score_stem = sum(resultats['stem_valide']) / len(resultats['stem_valide']) * 100
        
        return {
            'details': pd.DataFrame(resultats),
            'score_lisibilite_lemme': round(score_lemme, 1),
            'score_lisibilite_stem': round(score_stem, 1)
        }
    
    def _mot_francais_valide(self, mot: str) -> bool:
        """
        Heuristique simple pour vérifier si un mot ressemble à du français valide.
        
        Args:
            mot: Mot à vérifier
        
        Returns:
            True si le mot semble valide
        """
        # Critères basiques de validité
        if len(mot) < 2:
            return False
        
        # Pas trop de consonnes consécutives
        consonnes_consecutives = 0
        voyelles = 'aeiouéèêëàâäîïôöùûüÿy'
        
        for char in mot.lower():
            if char in voyelles:
                consonnes_consecutives = 0
            else:
                consonnes_consecutives += 1
                if consonnes_consecutives > 3:
                    return False
        
        # Doit contenir au moins une voyelle
        if not any(char in voyelles for char in mot.lower()):
            return False
        
        return True
    
    def rapport_complet(self, texte: str) -> None:
        """
        Génère un rapport de comparaison complet.
        
        Args:
            texte: Texte pour les tests
        """
        print("⚔️ RAPPORT COMPARATIF COMPLET\n")
        print(f"📝 Texte testé : {texte[:100]}...\n")
        
        # Test de performance
        perf = self.test_performance(texte, nb_iterations=5)
        print("🚀 PERFORMANCE :")
        print(f"  • Lemmatisation : {perf['temps_lemmatisation']*1000:.1f} ms")
        print(f"  • Stemming : {perf['temps_stemming']*1000:.1f} ms")
        if perf['ratio_vitesse'] > 0:
            print(f"  • Stemming {perf['ratio_vitesse']:.1f}x plus rapide")
        
        # Test de qualité
        qualite = self.test_qualite(texte)
        print(f"\n📊 QUALITÉ :")
        print(f"  • Vocabulaire original : {qualite['mots_originaux']} mots uniques")
        print(f"  • Après lemmatisation : {qualite['vocabulaire_lemmatisation']} ({qualite['reduction_lemmatisation']}% réduction)")
        print(f"  • Après stemming : {qualite['vocabulaire_stemming']} ({qualite['reduction_stemming']}% réduction)")
        
        print(f"\n🔍 ÉCHANTILLONS :")
        print(f"  • Lemmes : {' '.join(qualite['echantillon_lemmes'])}")
        print(f"  • Stems : {' '.join(qualite['echantillon_stems'])}")

# Test comparatif complet
comparateur = ComparateurLemmaStem()

texte_test_complet = """
Les musiciens jouaient magnifiquement dans la salle de concert. 
Les spectateurs applaudissaient enthusiastement après chaque morceau. 
Les instrumentistes démontrer une maîtrise technique exceptionnelle. 
Cette soirée musicale restera mémorable pour tous les participants.
"""

comparateur.rapport_complet(texte_test_complet)

In [ ]:
# Test de lisibilité détaillé
mots_lisibilite = [
    "magnifiquement", "rapidement", "courions", "finissaient",
    "musiciens", "spectateurs", "applaudissaient", "démontrer",
    "exceptionnelle", "mémorable", "participants", "instrumentistes"
]

if comparateur.lemmatiseur.modele_charge and comparateur.stemmer.snowball_disponible:
    test_lisibilite = comparateur.test_lisibilite(mots_lisibilite)
    
    print("\n👁️ TEST DE LISIBILITÉ\n")
    print("📋 Résultats détaillés :")
    
    # Affichage du tableau
    df_lisibilite = test_lisibilite['details']
    df_affichage = df_lisibilite[['mot', 'lemme', 'stem', 'lemme_valide', 'stem_valide']].copy()
    
    # Formatage pour l'affichage
    df_affichage['lemme_valide'] = df_affichage['lemme_valide'].map({True: '✅', False: '❌'})
    df_affichage['stem_valide'] = df_affichage['stem_valide'].map({True: '✅', False: '❌'})
    
    print(df_affichage.to_string(index=False))
    
    print(f"\n📊 Scores de lisibilité :")
    print(f"  • Lemmatisation : {test_lisibilite['score_lisibilite_lemme']}% de mots valides")
    print(f"  • Stemming : {test_lisibilite['score_lisibilite_stem']}% de mots valides")
    
    # Recommandation
    if test_lisibilite['score_lisibilite_lemme'] > test_lisibilite['score_lisibilite_stem']:
        print(f"\n💡 Recommandation : La lemmatisation produit des résultats plus lisibles")
    else:
        print(f"\n💡 Recommandation : Le stemming donne des résultats acceptables")
else:
    print("⚠️ Test de lisibilité non disponible (spaCy ou NLTK requis)")

## 6. 🎯 Cas Difficiles et Solutions

Comment gérer les cas complexes de la lemmatisation française ?

In [ ]:
# Cas difficiles spécifiques au français
cas_difficiles = {
    "Verbes irréguliers": {
        "mots": ["va", "vont", "irai", "iras", "suis", "es", "est", "sommes", "êtes", "sont"],
        "lemmes_attendus": ["aller", "aller", "aller", "aller", "être", "être", "être", "être", "être", "être"],
        "difficulte": "Formes radicalement différentes du lemme"
    },
    
    "Participes passés irréguliers": {
        "mots": ["pris", "prise", "prises", "mis", "mise", "mises", "fait", "faite", "faites"],
        "lemmes_attendus": ["prendre", "prendre", "prendre", "mettre", "mettre", "mettre", "faire", "faire", "faire"],
        "difficulte": "Participes très différents de l'infinitif"
    },
    
    "Homonymie contextuelle": {
        "mots": ["fils", "fils", "livre", "livre", "rose", "rose"],
        "contextes": ["le fils de métal", "mon fils", "un livre rouge", "il livre demain", "une rose rouge", "elle rose"],
        "lemmes_attendus": ["fil", "fils", "livre", "livrer", "rose", "roser"],
        "difficulte": "Même forme, sens différents selon contexte"
    },
    
    "Mots composés": {
        "mots": ["aujourd'hui", "c'est-à-dire", "peut-être", "quelqu'un", "rendez-vous"],
        "lemmes_attendus": ["aujourd'hui", "c'est-à-dire", "peut-être", "quelqu'un", "rendez-vous"],
        "difficulte": "Gestion des apostrophes et traits d'union"
    }
}

def tester_cas_difficiles() -> None:
    """
    Teste la lemmatisation sur les cas difficiles du français.
    """
    print("🎯 CAS DIFFICILES DE LA LEMMATISATION FRANÇAISE\n")
    
    if not lemmatiseur.modele_charge:
        print("⚠️ Tests non disponibles (spaCy requis)")
        return
    
    for categorie, info in cas_difficiles.items():
        print(f"📂 {categorie.upper()}")
        print(f"   💡 Difficulté : {info['difficulte']}")
        print(f"   🧪 Tests :")
        
        # Cas avec contexte (homonymie)
        if 'contextes' in info:
            for i, contexte in enumerate(info['contextes']):
                attendu = info['lemmes_attendus'][i]
                
                # Analyse du contexte complet
                doc = nlp_fr(contexte)
                mot_cible = info['mots'][i]
                
                # Trouver le lemme du mot cible dans le contexte
                lemme_obtenu = "Non trouvé"
                for token in doc:
                    if token.text.lower() == mot_cible.lower():
                        lemme_obtenu = token.lemma_
                        break
                
                # Vérification
                if lemme_obtenu.lower() == attendu.lower():
                    status = "✅"
                else:
                    status = "❌"
                
                print(f"      '{mot_cible}' dans '{contexte}' → {lemme_obtenu} (attendu: {attendu}) {status}")
        
        # Cas normaux
        else:
            for i, mot in enumerate(info['mots']):
                attendu = info['lemmes_attendus'][i]
                lemme_obtenu = lemmatiseur.lemmatiser_mot(mot)
                
                # Vérification
                if lemme_obtenu.lower() == attendu.lower():
                    status = "✅"
                else:
                    status = "❌"
                
                print(f"      {mot:12s} → {lemme_obtenu:12s} (attendu: {attendu:10s}) {status}")
        
        print()

# Test des cas difficiles
tester_cas_difficiles()

In [ ]:
# Solutions pour améliorer la lemmatisation
class LemmatiseurAmeliore:
    """Lemmatiseur avec corrections pour les cas difficiles français."""
    
    def __init__(self):
        self.lemmatiseur_base = LemmatiseurSpacy()
        
        # Dictionnaire de corrections manuelles
        self.corrections_manuelles = {
            # Verbes irréguliers problématiques
            'va': 'aller',
            'vont': 'aller',
            'irai': 'aller',
            'iras': 'aller',
            'suis': 'être',  # ambiguë avec "suivre"
            'fus': 'être',
            
            # Participes passés difficiles
            'pris': 'prendre',
            'prise': 'prendre',
            'prises': 'prendre',
            'mis': 'mettre',
            'mise': 'mettre',
            'mises': 'mettre',
            
            # Mots composés
            "aujourd'hui": "aujourd'hui",
            "c'est-à-dire": "c'est-à-dire",
            "peut-être": "peut-être"
        }
        
        # Règles contextuelles pour l'homonymie
        self.regles_contextuelles = {
            'fils': {
                'fil': ['métal', 'coton', 'couture', 'tissu'],
                'fils': ['enfant', 'famille', 'papa', 'maman']
            },
            'livre': {
                'livre': ['page', 'roman', 'lecture', 'bibliothèque'],
                'livrer': ['commande', 'colis', 'transport', 'demain']
            }
        }
    
    def lemmatiser_avec_contexte(self, texte: str) -> List[str]:
        """
        Lemmatise en tenant compte du contexte et des corrections.
        
        Args:
            texte: Texte à lemmatiser
        
        Returns:
            Liste des lemmes corrigés
        """
        if not self.lemmatiseur_base.modele_charge:
            return texte.split()
        
        # Analyse avec spaCy
        doc = nlp_fr(texte)
        lemmes_corriges = []
        
        for i, token in enumerate(doc):
            if token.is_punct or token.is_space:
                continue
            
            mot = token.text.lower()
            lemme_base = token.lemma_.lower()
            
            # 1. Vérifier les corrections manuelles
            if mot in self.corrections_manuelles:
                lemme_final = self.corrections_manuelles[mot]
            
            # 2. Appliquer les règles contextuelles
            elif mot in self.regles_contextuelles:
                lemme_final = self._resoudre_homonymie(mot, doc, i)
            
            # 3. Utiliser le lemme spaCy par défaut
            else:
                lemme_final = lemme_base
            
            lemmes_corriges.append(lemme_final)
        
        return lemmes_corriges
    
    def _resoudre_homonymie(self, mot: str, doc, position: int) -> str:
        """
        Résout l'homonymie en analysant le contexte.
        
        Args:
            mot: Mot ambigu
            doc: Document spaCy
            position: Position du mot dans le document
        
        Returns:
            Lemme le plus probable
        """
        if mot not in self.regles_contextuelles:
            return doc[position].lemma_.lower()
        
        # Extraire le contexte (mots environnants)
        contexte_mots = []
        fenetre = 3  # Mots avant et après
        
        for i in range(max(0, position - fenetre), 
                      min(len(doc), position + fenetre + 1)):
            if i != position and not doc[i].is_punct:
                contexte_mots.append(doc[i].text.lower())
        
        # Calculer les scores pour chaque sens possible
        scores = {}
        for sens, mots_cles in self.regles_contextuelles[mot].items():
            score = sum(1 for mot_cle in mots_cles 
                       if any(mot_cle in mot_contexte for mot_contexte in contexte_mots))
            scores[sens] = score
        
        # Retourner le sens avec le meilleur score
        if scores:
            meilleur_sens = max(scores, key=scores.get)
            return meilleur_sens
        
        # Par défaut, utiliser spaCy
        return doc[position].lemma_.lower()
    
    def comparer_avec_base(self, texte: str) -> Dict:
        """
        Compare les résultats avec et sans améliorations.
        
        Args:
            texte: Texte de test
        
        Returns:
            Dictionnaire avec la comparaison
        """
        lemmes_base = self.lemmatiseur_base.lemmatiser_texte(texte)
        lemmes_ameliores = self.lemmatiser_avec_contexte(texte)
        
        # Identifier les différences
        differences = []
        for i, (base, ameliore) in enumerate(zip(lemmes_base, lemmes_ameliores)):
            if base != ameliore:
                differences.append({
                    'position': i,
                    'lemme_base': base,
                    'lemme_ameliore': ameliore
                })
        
        return {
            'lemmes_base': lemmes_base,
            'lemmes_ameliores': lemmes_ameliores,
            'nb_differences': len(differences),
            'differences': differences
        }

# Test du lemmatiseur amélioré
lemmatiseur_ameliore = LemmatiseurAmeliore()

textes_test_amelioration = [
    "Je vais chez mon fils avec un livre.",
    "Il livre le fils de métal aujourd'hui.",
    "Les roses sont prises dans le livre.",
    "Mon fils a pris le livre et va le livrer."
]

print("🚀 TEST DU LEMMATISEUR AMÉLIORÉ\n")

if lemmatiseur_ameliore.lemmatiseur_base.modele_charge:
    for i, texte in enumerate(textes_test_amelioration, 1):
        print(f"📝 Test {i}: {texte}")
        
        comparaison = lemmatiseur_ameliore.comparer_avec_base(texte)
        
        print(f"   Base     : {' '.join(comparaison['lemmes_base'])}")
        print(f"   Amélioré : {' '.join(comparaison['lemmes_ameliores'])}")
        
        if comparaison['nb_differences'] > 0:
            print(f"   🔧 {comparaison['nb_differences']} amélioration(s) détectée(s)")
            for diff in comparaison['differences']:
                print(f"      '{diff['lemme_base']}' → '{diff['lemme_ameliore']}'")
        else:
            print(f"   ✅ Aucune amélioration nécessaire")
        print()
else:
    print("⚠️ Test du lemmatiseur amélioré non disponible (spaCy requis)")

## 7. 📊 Tests et Évaluation

Comment évaluer la qualité de votre lemmatisation ?

In [ ]:
# Framework d'évaluation pour la lemmatisation
class EvaluateurLemmatisation:
    """Évalue la qualité de la lemmatisation française."""
    
    def __init__(self):
        # Jeu de test avec vérité terrain
        self.jeu_test = {
            # Verbes réguliers
            'mangeons': 'manger',
            'mangeait': 'manger',
            'finissons': 'finir',
            'finissait': 'finir',
            
            # Verbes irréguliers
            'va': 'aller',
            'vont': 'aller',
            'suis': 'être',  # contexte dépendant
            'sommes': 'être',
            
            # Noms
            'chats': 'chat',
            'animaux': 'animal',
            'chevaux': 'cheval',
            
            # Adjectifs
            'beaux': 'beau',
            'belles': 'beau',
            'grandes': 'grand',
            
            # Participes passés
            'mangées': 'manger',
            'finies': 'finir',
            'prises': 'prendre'
        }
    
    def evaluer_lemmatiseur(self, lemmatiseur, nom: str = "Lemmatiseur") -> Dict:
        """
        Évalue un lemmatiseur sur le jeu de test.
        
        Args:
            lemmatiseur: Objet avec méthode lemmatiser_mot
            nom: Nom du lemmatiseur pour l'affichage
        
        Returns:
            Dictionnaire avec les métriques d'évaluation
        """
        resultats = []
        correct = 0
        
        for mot, lemme_attendu in self.jeu_test.items():
            if hasattr(lemmatiseur, 'lemmatiser_mot'):
                lemme_obtenu = lemmatiseur.lemmatiser_mot(mot)
            elif hasattr(lemmatiseur, 'stem'):
                lemme_obtenu = lemmatiseur.stem(mot)
            else:
                lemme_obtenu = mot
            
            est_correct = lemme_obtenu.lower() == lemme_attendu.lower()
            if est_correct:
                correct += 1
            
            resultats.append({
                'mot': mot,
                'attendu': lemme_attendu,
                'obtenu': lemme_obtenu,
                'correct': est_correct
            })
        
        precision = correct / len(self.jeu_test) * 100
        
        return {
            'nom': nom,
            'precision': round(precision, 1),
            'correct': correct,
            'total': len(self.jeu_test),
            'details': resultats
        }
    
    def comparer_lemmatiseurs(self, lemmatiseurs: Dict) -> None:
        """
        Compare plusieurs lemmatiseurs.
        
        Args:
            lemmatiseurs: Dictionnaire {nom: lemmatiseur}
        """
        print("📊 ÉVALUATION COMPARATIVE DES LEMMATISEURS\n")
        
        resultats_globaux = []
        
        for nom, lemmatiseur in lemmatiseurs.items():
            evaluation = self.evaluer_lemmatiseur(lemmatiseur, nom)
            resultats_globaux.append(evaluation)
            
            print(f"🔧 {nom.upper()}")
            print(f"   Précision : {evaluation['precision']}% ({evaluation['correct']}/{evaluation['total']})")
            
            # Afficher quelques erreurs
            erreurs = [r for r in evaluation['details'] if not r['correct']]
            if erreurs:
                print(f"   Erreurs : {', '.join([f"{e['mot']}→{e['obtenu']}" for e in erreurs[:3]])}...")
            print()
        
        # Classement
        resultats_globaux.sort(key=lambda x: x['precision'], reverse=True)
        print("🏆 CLASSEMENT :")
        for i, result in enumerate(resultats_globaux, 1):
            print(f"   {i}. {result['nom']} : {result['precision']}%")
    
    def analyser_erreurs(self, evaluation: Dict) -> None:
        """
        Analyse détaillée des erreurs.
        
        Args:
            evaluation: Résultat d'évaluation
        """
        erreurs = [r for r in evaluation['details'] if not r['correct']]
        
        if not erreurs:
            print(f"✅ Aucune erreur détectée pour {evaluation['nom']}")
            return
        
        print(f"🔍 ANALYSE DES ERREURS - {evaluation['nom'].upper()}\n")
        
        # Classification des erreurs
        types_erreurs = {
            'verbes_irreguliers': ['va', 'vont', 'suis', 'sommes'],
            'pluriels_irreguliers': ['animaux', 'chevaux'],
            'accords_participes': ['mangées', 'finies', 'prises'],
            'adjectifs_irreguliers': ['beaux', 'belles']
        }
        
        erreurs_par_type = defaultdict(list)
        
        for erreur in erreurs:
            mot = erreur['mot']
            type_trouve = 'autres'
            
            for type_erreur, mots_type in types_erreurs.items():
                if mot in mots_type:
                    type_trouve = type_erreur
                    break
            
            erreurs_par_type[type_trouve].append(erreur)
        
        # Affichage par type
        for type_erreur, erreurs_type in erreurs_par_type.items():
            print(f"📂 {type_erreur.replace('_', ' ').title()} ({len(erreurs_type)} erreurs) :")
            for erreur in erreurs_type:
                print(f"   '{erreur['mot']}' → '{erreur['obtenu']}' (attendu: '{erreur['attendu']}')")
            print()

# Test d'évaluation
evaluateur = EvaluateurLemmatisation()

# Préparation des lemmatiseurs à tester
lemmatiseurs_test = {}

if lemmatiseur.modele_charge:
    lemmatiseurs_test['spaCy'] = lemmatiseur

if lemmatiseur_ameliore.lemmatiseur_base.modele_charge:
    lemmatiseurs_test['spaCy Amélioré'] = lemmatiseur_ameliore

if stemmer.snowball_disponible:
    lemmatiseurs_test['Stemming Snowball'] = stemmer

if lemmatiseurs_test:
    # Comparaison des lemmatiseurs
    evaluateur.comparer_lemmatiseurs(lemmatiseurs_test)
    
    # Analyse détaillée des erreurs pour le meilleur lemmatiseur
    if 'spaCy' in lemmatiseurs_test:
        evaluation_spacy = evaluateur.evaluer_lemmatiseur(lemmatiseurs_test['spaCy'], 'spaCy')
        print("\n" + "="*60)
        evaluateur.analyser_erreurs(evaluation_spacy)
else:
    print("⚠️ Aucun lemmatiseur disponible pour l'évaluation")

In [ ]:
# Visualisation des résultats d'évaluation
if lemmatiseurs_test:
    # Collecte des données pour visualisation
    donnees_viz = []
    
    for nom, lemmatiseur in lemmatiseurs_test.items():
        evaluation = evaluateur.evaluer_lemmatiseur(lemmatiseur, nom)
        donnees_viz.append({
            'Lemmatiseur': nom,
            'Précision (%)': evaluation['precision'],
            'Correct': evaluation['correct'],
            'Erreurs': evaluation['total'] - evaluation['correct']
        })
    
    df_viz = pd.DataFrame(donnees_viz)
    
    # Création des graphiques
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Graphique 1 : Précision par lemmatiseur
    bars1 = axes[0].bar(df_viz['Lemmatiseur'], df_viz['Précision (%)'], 
                        color=['lightblue', 'lightgreen', 'lightcoral'][:len(df_viz)], 
                        alpha=0.7, edgecolor='darkblue')
    axes[0].set_title('📊 Précision des Lemmatiseurs', fontweight='bold', fontsize=14)
    axes[0].set_ylabel('Précision (%)')
    axes[0].set_ylim(0, 100)
    axes[0].tick_params(axis='x', rotation=45)
    
    # Ajout des valeurs sur les barres
    for bar, value in zip(bars1, df_viz['Précision (%)']):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                     f'{value}%', ha='center', va='bottom', fontweight='bold')
    
    # Graphique 2 : Répartition correct/erreurs
    x = np.arange(len(df_viz))
    width = 0.35
    
    bars2 = axes[1].bar(x - width/2, df_viz['Correct'], width, 
                        label='Correct', color='lightgreen', alpha=0.7)
    bars3 = axes[1].bar(x + width/2, df_viz['Erreurs'], width, 
                        label='Erreurs', color='lightcoral', alpha=0.7)
    
    axes[1].set_title('📈 Répartition Correct/Erreurs', fontweight='bold', fontsize=14)
    axes[1].set_ylabel('Nombre de mots')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(df_viz['Lemmatiseur'], rotation=45)
    axes[1].legend()
    
    # Ajout des valeurs
    for bars in [bars2, bars3]:
        for bar in bars:
            height = bar.get_height()
            axes[1].text(bar.get_x() + bar.get_width()/2., height + 0.1,
                         f'{int(height)}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Tableau récapitulatif
    print("\n📋 TABLEAU RÉCAPITULATIF :")
    print(df_viz.to_string(index=False))
else:
    print("⚠️ Aucune donnée pour la visualisation")

## 8. 💡 Recommandations Pratiques

Quand utiliser quoi et comment optimiser vos choix ?

In [ ]:
# Guide de recommandations pratiques
recommandations = {
    "Analyse de sentiment": {
        "recommandation": "Lemmatisation spaCy",
        "raison": "Précision importante pour capturer les nuances émotionnelles",
        "exemple": "'magnifiques' → 'magnifique' vs 'magnif' (stemming)",
        "performance": "Élevée",
        "cas_usage": ["Avis clients", "Réseaux sociaux", "Satisfaction"]
    },
    
    "Classification de documents": {
        "recommandation": "Stemming Snowball",
        "raison": "Réduction maximale du vocabulaire, vitesse élevée",
        "exemple": "Plus de compression pour machine learning",
        "performance": "Très élevée",
        "cas_usage": ["Catégorisation automatique", "Filtrage spam", "Clustering"]
    },
    
    "Recherche d'information": {
        "recommandation": "Lemmatisation + Stemming hybride",
        "raison": "Équilibre entre précision et rappel",
        "exemple": "Rechercher 'course' doit trouver 'courions', 'coureur'",
        "performance": "Moyenne",
        "cas_usage": ["Moteurs de recherche", "FAQ automatisées", "Documentation"]
    },
    
    "Analyse syntaxique/sémantique": {
        "recommandation": "Lemmatisation spaCy avancée",
        "raison": "Préservation du sens et de la structure grammaticale",
        "exemple": "Analyse des relations sujet-verbe-complément",
        "performance": "Moyenne",
        "cas_usage": ["Extraction d'entités", "Résumé automatique", "Traduction"]
    },
    
    "Gros volumes (Big Data)": {
        "recommandation": "Stemming simple optimisé",
        "raison": "Vitesse primordiale sur millions de documents",
        "exemple": "Traitement temps réel de flux Twitter",
        "performance": "Très élevée",
        "cas_usage": ["Streaming", "Logs", "Monitoring"]
    },
    
    "Interface utilisateur": {
        "recommandation": "Lemmatisation spaCy",
        "raison": "Résultats lisibles et compréhensibles",
        "exemple": "Suggestions de recherche avec mots valides",
        "performance": "Moyenne",
        "cas_usage": ["Autocomplétion", "Suggestions", "Corrections"]
    }
}

def afficher_recommandations() -> None:
    """
    Affiche le guide des recommandations pratiques.
    """
    print("💡 GUIDE DES RECOMMANDATIONS PRATIQUES\n")
    
    for contexte, info in recommandations.items():
        print(f"📂 {contexte.upper()}")
        print(f"   🎯 Recommandation : {info['recommandation']}")
        print(f"   💭 Raison : {info['raison']}")
        print(f"   📝 Exemple : {info['exemple']}")
        print(f"   ⚡ Performance : {info['performance']}")
        print(f"   🔧 Cas d'usage : {', '.join(info['cas_usage'])}")
        print()

afficher_recommandations()

In [ ]:
# Outil d'aide à la décision
def aide_decision_lemmatisation(contexte: str = None, 
                               priorite_vitesse: bool = False,
                               priorite_precision: bool = False,
                               volume_donnees: str = "moyen") -> None:
    """
    Aide à choisir la meilleure approche de lemmatisation.
    
    Args:
        contexte: Contexte d'application
        priorite_vitesse: Si la vitesse est prioritaire
        priorite_precision: Si la précision est prioritaire
        volume_donnees: Volume de données ("petit", "moyen", "gros")
    """
    print("🧭 AIDE À LA DÉCISION : LEMMATISATION\n")
    
    # Analyse des critères
    score_lemma = 0
    score_stem = 0
    
    # Critère : priorité vitesse
    if priorite_vitesse:
        score_stem += 3
        print("⚡ Priorité vitesse → +3 points pour Stemming")
    
    # Critère : priorité précision
    if priorite_precision:
        score_lemma += 3
        print("🎯 Priorité précision → +3 points pour Lemmatisation")
    
    # Critère : volume de données
    if volume_donnees == "gros":
        score_stem += 2
        print("📊 Gros volume → +2 points pour Stemming")
    elif volume_donnees == "petit":
        score_lemma += 1
        print("📊 Petit volume → +1 point pour Lemmatisation")
    
    # Critère : contexte spécifique
    contextes_lemma = ["sentiment", "interface", "sémantique"]
    contextes_stem = ["classification", "clustering", "big data"]
    
    if contexte:
        if any(ctx in contexte.lower() for ctx in contextes_lemma):
            score_lemma += 2
            print(f"🎪 Contexte '{contexte}' → +2 points pour Lemmatisation")
        elif any(ctx in contexte.lower() for ctx in contextes_stem):
            score_stem += 2
            print(f"🎪 Contexte '{contexte}' → +2 points pour Stemming")
    
    # Décision finale
    print(f"\n📊 SCORES FINAUX :")
    print(f"   🌱 Lemmatisation : {score_lemma} points")
    print(f"   ✂️ Stemming : {score_stem} points")
    
    if score_lemma > score_stem:
        print(f"\n🏆 RECOMMANDATION : Lemmatisation (spaCy)")
        print(f"   💡 Pourquoi : Meilleure précision pour votre contexte")
        if lemmatiseur.modele_charge:
            print(f"   🛠️ Implémentation : Utilisez LemmatiseurSpacy()")
    elif score_stem > score_lemma:
        print(f"\n🏆 RECOMMANDATION : Stemming (Snowball)")
        print(f"   💡 Pourquoi : Vitesse et efficacité pour votre contexte")
        if stemmer.snowball_disponible:
            print(f"   🛠️ Implémentation : Utilisez StemmerFrancais()")
    else:
        print(f"\n🤝 RECOMMANDATION : Approche hybride")
        print(f"   💡 Pourquoi : Vos critères sont équilibrés")
        print(f"   🛠️ Implémentation : Testez les deux et choisissez selon les résultats")

# Tests de l'aide à la décision
print("🧪 TESTS DE L'AIDE À LA DÉCISION\n")

# Cas 1 : Analyse de sentiment
print("💬 Cas 1 : Analyse de sentiment d'avis clients")
aide_decision_lemmatisation(
    contexte="analyse sentiment",
    priorite_precision=True,
    volume_donnees="moyen"
)

print("\n" + "="*60 + "\n")

# Cas 2 : Classification de documents
print("📋 Cas 2 : Classification automatique de millions de documents")
aide_decision_lemmatisation(
    contexte="classification big data",
    priorite_vitesse=True,
    volume_donnees="gros"
)

print("\n" + "="*60 + "\n")

# Cas 3 : Interface utilisateur
print("🖥️ Cas 3 : Suggestions de recherche pour interface utilisateur")
aide_decision_lemmatisation(
    contexte="interface recherche",
    priorite_precision=True,
    volume_donnees="petit"
)

## 🎉 Conclusion et Synthèse

### ✅ Ce que vous avez appris :

1. **🤔 Concepts fondamentaux** : Différence claire entre lemmatisation et stemming
2. **🇫🇷 Spécificités françaises** : Verbes irréguliers, homonymie, accords complexes
3. **🚀 Outils pratiques** : Maîtrise de spaCy et NLTK pour le français
4. **⚔️ Comparaisons objectives** : Performance, qualité, cas d'usage
5. **🎯 Gestion des cas difficiles** : Solutions pour améliorer la précision
6. **📊 Évaluation rigoureuse** : Métriques et benchmarks pour valider vos choix
7. **💡 Aide à la décision** : Recommandations selon votre contexte

### 🏆 Recommandations finales :

#### 🌱 Utilisez la LEMMATISATION quand :
- L'**exactitude** est cruciale (analyse de sentiment, interface utilisateur)
- Vous voulez des **mots lisibles** par les humains
- Vous travaillez sur des **volumes modérés**
- Vous analysez le **sens** et la **sémantique**

#### ✂️ Utilisez le STEMMING quand :
- La **vitesse** est prioritaire (gros volumes, temps réel)
- Vous voulez une **réduction maximale** du vocabulaire
- Vous faites de la **classification** ou du **clustering**
- La précision exacte n'est **pas critique**

### 🔮 Prochaines étapes :

Maintenant que vous maîtrisez la lemmatisation française, vous êtes prêt pour :
- **Pipeline complet** intégrant nettoyage + tokenisation + lemmatisation
- **Représentation vectorielle** des mots (TF-IDF, Word2Vec)
- **Modèles avancés** de NLP français

---

**🎯 Message final :** La lemmatisation n'est pas qu'une étape technique, c'est un **choix stratégique** qui influence toute votre chaîne de traitement NLP. Choisissez consciemment selon votre contexte !